Code for San Francisco
# Clean Energy Challenge

Purpose¶
The purpose of this notebook is to create a quick model to predict townships that do not already have solar that are likely to be successful for solar investment

Looking for input variables in the areas of:
- Income - number of households who have income out of population (used employment as proxy)
- Payment systems available (too broad)
- Grid or alternative energy
- Market size, population

Output
- Yes/no boolean willing to pay
- Amount willing to pay
- Predicted success of solar

Proxy potentials
- Number of households who are already paying for light via non electricity (battery, candle) 
- How much households are paying for substitute light products by product
- Number of households who are already paying for communication channels via non electricity
- Number of households who are already paying for cooking utilies via non electricity

Preferred granularity
- Township
- Annual

In [ ]:
from __future__ import absolute_import, division, print_function
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os, sys

In [ ]:
#Read the CSV file 'Myanmar World Development Indicators"

#path_census = 'https://github.com/eayoungs/datasci-etech-chall/blob/master/data/HouseholdPopulationbaseddatasetMIMUTownshipsabbreviated.csv'
path_census = '/Users/eayoungs/repo/Code/Civic/C4SF/StateDept-EnergyChallenge/datasci-etech-chall/data/HouseholdPopulationbaseddatasetMIMUTownshipsabbreviated.csv'
df_census = pd.read_csv(path_census, header=0, index_col=0)

#path_labor = 'https://github.com/eayoungs/datasci-etech-chall/blob/master/data/HouseholdPopulationbaseddatasetMIMUTownshipsLabour.csv'
path_labor = '/Users/eayoungs/repo/Code/Civic/C4SF/StateDept-EnergyChallenge/datasci-etech-chall/data/HouseholdPopulationbaseddatasetMIMUTownshipsLabour.csv'
df_labor = pd.read_csv(path_labor, header=0, index_col=0)

#path_dictionary = 'https://github.com/eayoungs/datasci-etech-chall/blob/master/data/datadictionaryhhpoptownships.csv'
path_dictionary = '/Users/eayoungs/repo/Code/Civic/C4SF/StateDept-EnergyChallenge/datasci-etech-chall/data/datadictionaryhhpoptownships.csv'
df_dictionary = pd.read_csv(path_dictionary, header=0, index_col=0)

#path_indicators = 'https://github.com/eayoungs/datasci-etech-chall/blob/master/data/Myanmar%20world%20development%20indicators.csv'
path_indicators = '/Users/eayoungs/repo/Code/Civic/C4SF/StateDept-EnergyChallenge/datasci-etech-chall/data/Myanmar world development indicators.csv'
df_indicators = pd.read_csv(path_indicators, header=0, index_col=0)

In [ ]:
%matplotlib inline
df_census.light_t.hist()

In [ ]:
df_census.light_elec.hist()

In [ ]:
df_census.light_sol.hist()

In [ ]:
df_light = df_census[[
'pcode_ts',
'pop_hh',
'pop_ins',
'light_t',
'light_elec',
'light_kero',
'light_cand',
'light_batt',
'light_gen',
'light_wat',
'light_sol',
'light_oth']]

In [ ]:
df_light['light_substitute'] = df_light['light_kero'] + df_light['light_cand'] + df_light['light_batt'] + df_light['light_gen'] + df_light['light_wat'] + df_light['light_oth']
df_light['light_substitute_rate'] = df_light['light_substitute'] / df_light['pop_hh']
df_light['light_solar_rate'] = df_light['light_sol'] / df_light['pop_hh']
df_light['light_elec_rate'] = df_light['light_elec'] / df_light['pop_hh']


In [ ]:
df_light.light_substitute_rate.hist()

In [ ]:
df_light.light_solar_rate.hist()

In [ ]:
df_labor

# Linear regression model

### Inputs:
- Light_elec_rate as a proxy for already on grid
- pop_hh for population
- Employment

### Potential additional inputs:
- Female/male headed households
- Household size
- Housing Type
- Transportation
- Education
- Urban/Rural
- Source of drinking water
- Type of ownership

In [ ]:
import scipy.stats as stats
import sklearn
from sklearn.linear_model import LinearRegression

In [ ]:
df_model = df_light[[
'pop_hh',
'light_elec_rate',
'light_solar_rate']]

In [ ]:
df_model_indx = df_model.reset_index()
df_model_indx = df_model_indx[['pop_hh', 'light_elec_rate','light_solar_rate']]

In [ ]:
df_model_indx

In [ ]:
X = df_model_indx.drop('light_solar_rate', axis = 1)
y = df_model_indx[['light_solar_rate']]
y = y.round(decimals=5)
y = y.fillna(value=0)
lm = LinearRegression()
lm

In [ ]:
lm.fit(X, y)

In [ ]:
lm.coef_

In [ ]:
lm.intercept_

In [ ]:
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

In [ ]:
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

### Element	Description
- R-squared	The coefficient of determination. A statistical measure of how well the regression line approximates the real data points
- Adj. R-squared	The above value adjusted based on the number of observations and the degrees-of-freedom of the residuals
- F-statistic	A measure how significant the fit is. The mean squared error of the model divided by the mean squared error of the residuals
- Prob (F-statistic)	The probability that you would get the above statistic, given the null hypothesis that they are unrelated
- Log-likelihood	The log of the likelihood function.
- AIC	The Akaike Information Criterion. Adjusts the log-likelihood based on the number of observations and the complexity of the model.
- BIC	The Bayesian Information Criterion. Similar to the AIC, but has a higher penalty for models with more parameters.
- The second table reports for each of the coefficients
- coef	The estimated value of the coefficient
- std err	The basic standard error of the estimate of the coefficient. More sophisticated errors are also available.
- t	The t-statistic value. This is a measure of how statistically significant the coefficient is.
- P > |t|	P-value that the null-hypothesis that the coefficient = 0 is true. If it is less than the confidence level, often 0.05, it indicates that there is a statistically significant relationship between the term and the response.
- [95.0% Conf. Interval]	The lower and upper values of the 95% confidence interval

In [ ]:
plt.scatter(df_model_indx[['pop_hh']], Y)
plt.xlabel('Household Population Count')
plt.ylabel('Solar Light Rate')

In [ ]:
plt.scatter(df_model_indx[['light_elec_rate']], Y)
plt.xlabel('Electric Grid Rate')
plt.ylabel('Solar Light Rate')

## Add more variables
usuact_10ab_t
usuact_10ab_govemp_t
usuact_10ab_priemp_t
usuact_10ab_empyr_t
usuact_10ab_ownacc_t
usuact_10ab_unpfam_t
usuact_10ab_seekw_t
usuact_10ab_nseekw_t
usuact_10ab_stu_t
usuact_10ab_hhwork_t
usuact_10ab_retir_t
usuact_10ab_ill_t

In [ ]:
df_census['light_substitute'] = df_census['light_kero'] + df_census['light_cand'] + df_census['light_batt'] + df_census['light_gen'] + df_census['light_wat'] + df_census['light_oth']
df_census['light_substitute_rate'] = df_census['light_substitute'] / df_census['pop_hh']
df_census['light_solar_rate'] = df_census['light_sol'] / df_census['pop_hh']
df_census['light_elec_rate'] = df_census['light_elec'] / df_census['pop_hh']
df_census['sum_pop_hh'] = 47929999
df_census['pop_hh_rate'] = df_census['pop_hh'] / df_census['sum_pop_hh']
df_census['hh_m_rate'] = df_census['hh_m'] / df_census['pop_hh']
df_census['hh_f_rate'] = df_census['hh_f'] / df_census['pop_hh']
df_census['pop_u_rate'] = df_census['pop_u'] / df_census['pop_hh']
df_census['pop_r_rate'] = df_census['pop_r'] / df_census['pop_hh']
df_census['govemp_rate'] = df_census['usuact_10ab_govemp_t'] / df_census ['usuact_10ab_t']
df_census['priemp_rate'] = df_census['usuact_10ab_priemp_t'] / df_census['usuact_10ab_t']
df_census['empyr_rate'] = df_census['usuact_10ab_empyr_t'] / df_census['usuact_10ab_t']
df_census['ownacc_rate'] = df_census['usuact_10ab_ownacc_t'] / df_census['usuact_10ab_t']
df_census['unpfam_rate'] = df_census['usuact_10ab_unpfam_t'] / df_census['usuact_10ab_t']
df_census['seekw_rate'] = df_census['usuact_10ab_seekw_t'] / df_census['usuact_10ab_t']
df_census['stu_rate'] = df_census['usuact_10ab_stu_t'] / df_census['usuact_10ab_t']
df_census['hhwork_rate'] = df_census['usuact_10ab_hhwork_t'] / df_census['usuact_10ab_t']
df_census['retir_rate'] = df_census['usuact_10ab_retir_t'] / df_census['usuact_10ab_t']
df_census['ill_rate'] = df_census['usuact_10ab_ill_t'] / df_census['usuact_10ab_t']

In [ ]:
df_model_more = df_census[[
'pcode_ts',
'pop_hh_rate',
'hh_m_rate',
'hh_f_rate',
'pop_r_rate',
'pop_u_rate',
'govemp_rate',
'priemp_rate',
'empyr_rate',
'ownacc_rate',
'unpfam_rate',
'seekw_rate',
'stu_rate',
'hhwork_rate',
'retir_rate',
'ill_rate',
'light_elec_rate',
'light_solar_rate',
'light_substitute_rate']]

In [ ]:
df_model_more
df_model_more = df_model_more.set_index(['pcode_ts'])

In [ ]:
df_model_more

In [ ]:
Xm = df_model_more.drop('light_solar_rate', axis = 1)
ym = df_model_more[['light_solar_rate']]
Xm = Xm.round(decimals=5)
Xm = Xm.fillna(value=0)
ym = ym.round(decimals=5)
ym = ym.fillna(value=0)
lm = LinearRegression()
lm

In [ ]:
lm.fit(Xm, ym)

In [ ]:
Xm2 = sm.add_constant(Xm)
estm = sm.OLS(ym, Xm2)
est2m = estm.fit()
print(est2m.summary())